In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
import spacy
def que_classification():
    f=open("/content/drive/My Drive/final_intern/predefined.txt","r",encoding='utf-8')
    classes=[] #all the labels of the questions
    query=[]   #queries corresponding to the classes
    new_label=[] #labels in numerical representation
    for line in f:
        line=line.rstrip('\n')
        classes.append((line.split()[0]).split(":")[0])  #appending labels from the test to classes list
        query.append(line[5:])                           #appending queries from text to query list
    j=list(set(classes))   #set of the labels
    k=[i for i in range(len(j))] #numbers for labels
    my_dict={}
    for i in range(7):
        my_dict.update({j[i]:k[i]}) #dictionary {label:number}
    for i in classes:
        new_label.append(my_dict[i]) #labels in numeric representation
    #query.append(given_query)
    #vectorising all the inputs
    vectorizer =TfidfVectorizer()
    #X = vectorizer.fit_transform(classes) #Learn vocabulary and idf, return document-term matrix (input-raw_document)
    Y = vectorizer.fit_transform(query)
    z=Y.toarray()
    #print(Y[-1])
    X = z[:len(new_label)]#y[:1342]
    y = new_label[:len(new_label)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1250)

    lr = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(14,), random_state=1)

    lr.fit(X_train,y_train)
    #print("Accuracy")
    return (vectorizer,lr,my_dict)
def predict_ques(que):
    #withbias=list(z[-1]) #question
    #withbias.insert(0,1) # adding bias at front
    query=[]
    query.append(que)
    withbias=vectorizer.transform(query).toarray()
    maxi= lr.predict(withbias)
    for i in my_dict:
            if my_dict[i]==maxi:
                print("\nLabel of the given query :",i)
                break
    return i
(vectorizer,lr,my_dict)=que_classification()

In [ ]:
import re, math
import urllib
from googletrans import Translator
import urllib3
import warnings
from bs4 import BeautifulSoup
from collections import Counter
import nltk
from nltk.tag.stanford import StanfordNERTagger
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import inflect
p = inflect.engine()
import string
import os
java_path = "C:/Program Files/Java/jdk1.8.0_151/bin/java.exe"
os.environ['JAVAHOME'] = java_path

warnings.filterwarnings('ignore')
prelink="https://www.bing.com/search?"
#from urllib.request import urlopen
def preprocess(x):
    # Step1 to lowercase
    lowercase=x.lower()
    temp_str = lowercase.split()
    # Step2 removing digits
    new_string = []
    p = inflect.engine()
    for word in temp_str:
        # if word is a digit, convert the digit
        # to numbers and append into the new_string list
        if word.isdigit():
            temp = p.number_to_words(word)
            new_string.append(temp)

        # append the word as it is
        else:
            new_string.append(word)

    # join the words of new_string to form a string
    temp_str = ' '.join(new_string)
    # Step-3 removing punctuations
    translator = str.maketrans('', '', string.punctuation)
    temp_str = temp_str.translate(translator)
    # Step-4 removing stop words
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(temp_str)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    filtered_text =' '.join(filtered_text)
    # step-5 Lemmatise
    lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(filtered_text)
    # provide context i.e. part-of-speech
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens]
    return lemmas
def NER_With_corenlp(qc):
    print("qc :",qc)
    from pycorenlp import StanfordCoreNLP
    nlp = StanfordCoreNLP('http://localhost:9000')
    Possible_answers = []
    All_answers      = []
    Ner_Entites_Dict ={'PERSON':[],'ORG':[],'CARDINAL':[],'TIME':[],'DATE':[],'GPE':[],'PERCENT':[],'NORP':[],'ORDINAL':[],'PRODUCT':[],'LAW':[]}
    for i in range(len(Top_similarity_sent)):
        while(1):
            try:
                result = nlp.annotate(Top_similarity_sent[i],
                           properties={
                               'annotators': 'sentiment, ner, pos',
                               'outputFormat': 'json',
                               'timeout': 1000,
                           })
                for word in result["sentences"][0]['tokens']:
                    Ner_Entites_Dict[word['word']]=word['ner']
                break
            except TypeError as t:
                print("TimeOut- Restarting")
    if qc in Ner_Entites_Dict.keys():
        return Ner_Entites_Dict[qc]


def NER_other_way(qc):
    print("qc : ",qc)

    nlp = spacy.load('en_core_web_sm')
    #stanford_ner_tagger = StanfordNERTagger('C:/Users/B151119/Desktop/SummerInternship/Week-5/stanford-ner-4.0.0/classifiers/english.muc.7class.distsim.crf.ser.gz','C:/Users/B151119/Desktop/SummerInternship/Week-5/stanford-ner-4.0.0/stanford-ner.jar')
    ners1 ={'PERSON':[],'ORG':[],'CARDINAL':[],'TIME':[],'DATE':[],'GPE':[],'PERCENT':[],'LAW':[],'ORDINAL'
        :[],'NORP':[],'LOC':[],'EVENT':[],'FAC':[],'QUANTITY':[],'MONEY':[]}
    for i in range(len(Top_similarity_sent)):
        sentence=Top_similarity_sent[i]
        doc = nlp(sentence)
        for ent in doc.ents:
            if(ent.label_ in ners1.keys()):
                ners1[ent.label_].append(ent.text)
    print(ners1)
    ners=ners1
    ners['LOCA'] = ners.pop('GPE')
    ners['ORGA'] = ners.pop('ORG')
    ners['NUMB'] = ners.pop('CARDINAL')
    ners['PERC'] = ners.pop('PERCENT')
    ners['PERS'] = ners.pop('PERSON')
    ners['CUR'] = ners.pop('MONEY')
    ners['QUANT']=ners.pop('QUANTITY')
    '''
    lis = ners['TIME']
    for i in lis:
        ners['NUMB'].append(i)
    lis = ners['PERC']
    for i in lis:
        ners['NUMB'].append(i)

    '''
    lis = ners['LOC']
    for i in lis:
        ners['LOCA'].append(i)
    #lis = ners['NORP']
    #for i in lis:
        #ners['PERS'].append(i)
    lis = ners['ORDINAL']
    for i in lis:
        ners['NUMB'].append(i)
    return (ners1,ners[qc])

    #print("Possible_answers : ",Possible_answers)
    #print("\n\n")
    #print("All_answers : ",All_answers)
    #print("\n\n")
    #return Possible_answers

def qquery(qu):
    # BING
    print("BING SEARCH")
    address = "http://www.bing.com/search?q=%s" % (urllib.parse.quote_plus(str(qu)))
    http = urllib3.PoolManager()
    resp = http.request('GET',address)
    soup = BeautifulSoup(resp.data)
    print("Top 10 URLS:\n----------------------------------------------------------------\n\n")
    if not(soup.findAll('li', attrs={'class':'b_algo'})):
        print("Not able to find urls, Some Error came call back the function again....\n")
        qquery(qu)

    for listt in soup.findAll('li', attrs={'class':'b_algo'}):
        head_links = listt.find('h2')
        for link in head_links.find_all('a', href=True):
            Top_10_urls.append(link['href'])
            print(link['href']+"\n")
            Whole_output.append(link['href']+"\n")
    #print("OK returning")
    #GOOGLE
    '''
    print("GOOGLE")
    try:
        from googlesearch import search
    except ImportError:
        print("No module named 'google' found")
    for j in search(qu, tld="co.in", num=10, stop=10, pause=2):
        Top_10_urls.append(j)
        print(j)
    '''

    return Top_10_urls


def translate(to_translate, to_langage="auto", langage="auto"):
    try:
        translator = Translator()
        result = translator.translate(to_translate, src='te', dest='en')
        quest.append(result.text)
        Whole_output.append(result.text)
        return result
    except:
        print("Translate error")
        return translate(to_translate, to_langage="auto", langage="auto")

def translate2(to_translate, to_langage="auto", langage="auto"):
    import goslate
    gs = goslate.Goslate()
    print(gs.translate(to_translate, 'en'))
    result = gs.translate(to_translate, 'en')
    return result

In [ ]:
def Sentence_Extraction():
    Numlinks=len(Top_10_urls)
    #sen=open("Whole_sentence.txt","w")
    i=0
    for i in range(Numlinks):
        try:
            nP=0
            http = urllib3.PoolManager()
            resp = http.request('GET',Top_10_urls[i])
            #content=urllib2.urlopen(Top_10_urls[i]) ############ May be Here you may get ERROR
            htmlSource=resp.data	############ May be Here you may get ERROR
            contentt = BeautifulSoup(htmlSource)

            for para in contentt.findAll(['p']):
                pa=para.text
                whole_sent.append(pa.strip()) #all sentence writing into one list line by line
                #sen.write(pa.encode('UTF-8')+"\n") # all sentence writing into one file line by line
                nP=nP+1 # number of imp content paragraphs in each URL...........
        except IOError as e:
            print(" IO ERROR ")
        except ValueError:
            print("VALUE ERROR ")
        except:
            print("ERROR ")
            Top_10_urls.pop(i)
            Sentence_Extraction()
            break

    #sen.close()
    #print("\n\nTotal extracted lines :"+str(len(whole_sent))+"\n\n")


def Finding_Similarity_Matrix(q):
    cc=0
    for i in range(len(whole_sent)):
        stmt=whole_sent[i].split(".")
        for j in range(len(stmt)):
            if(len(stmt[j].split("?"))==1):
                cc=cc+1
                all_sen.append(stmt[j])
    all_sen1=list(set(all_sen))
    WORD = re.compile(r'\w+')
    def get_cosine(vec1, vec2):
        intersection = set(vec1.keys()) & set(vec2.keys())
        numerator = sum([vec1[x] * vec2[x] for x in intersection])

        sum1 = sum([vec1[x]**2 for x in vec1.keys()])
        sum2 = sum([vec2[x]**2 for x in vec2.keys()])
        denominator = math.sqrt(sum1) * math.sqrt(sum2)

        if not denominator:
            return(0.0)
        else:
            return(float(numerator) / denominator)

    def text_to_vector(text):
        words = WORD.findall(text)
        return(Counter(words))
    text1 =q  # given user query
    x=0

    for n in range(len(all_sen)):
        text2=all_sen[n]
        vector1 = text_to_vector(text1)
        vector2 = text_to_vector(text2)

        cosine = get_cosine(vector1, vector2)
        x=x+1
        consine_list.append(cosine)


def Most_Relative_Sentence():

	num=10 ##number of most relative sentences considering
	if(len(whole_sent)<num):
		num = len(whole_sent)
	tt= (sorted(consine_list, reverse=True)[:num])
	for k in range(num):
		l=[i for i,j in enumerate(consine_list) if j==tt[k]]
		myString = all_sen[l[0]].strip()
		sent = ' '.join(myString.split())
		Top_similarity_sent.append(sent)
		Whole_output.append(sent+"\n")
	return Top_similarity_sent

def que_ner(qc_label,sent_list):
    ners1 ={'PERSON':[],'ORG':[],'CARDINAL':[],'TIME':[],'DATE':[],'GPE':[],'PERCENT':[],'LAW':[],'ORDINAL'
        :[],'NORP':[],'LOC':[],'EVENT':[],'FAC':[],'QUANTITY':[],'MONEY':[]}
    for i in range(len(sent_list)):
        sentence=sent_list[i]
        nlp = spacy.load('en_core_web_sm')
        doc = nlp(sentence)
        for ent in doc.ents:
            if(ent.label_ in ners1.keys()):
                ners1[ent.label_].append(ent.text)
    ners=ners1
    print(ners)
    ners['LOCA'] = ners.pop('GPE')
    ners['ORGA'] = ners.pop('ORG')
    ners['NUMB'] = ners.pop('CARDINAL')
    ners['PERC'] = ners.pop('PERCENT')
    ners['PERS'] = ners.pop('PERSON')
    ners['CUR'] = ners.pop('MONEY')
    ners['QUANT']=ners.pop('QUANTITY')
    lis = ners['LOC']
    for i in lis:
        ners['LOCA'].append(i)
    #lis = ners['NORP']
    #for i in lis:
        #ners['PERS'].append(i)
    lis = ners['ORDINAL']
    for i in lis:
        ners['NUMB'].append(i)
    for i in range(len(ners[qc_label])):
        ners[qc_label][i]=ners[qc_label][i].lower()
    result=ners[qc_label]
    return (ners1,result)

In [ ]:
############################################# Calling QA System Step by Step #########################################################
import pandas as pd
import time
input_file = open("/content/drive/My Drive/final_intern/updated_file_847_que","r",encoding='utf-8').readlines()
Questions = []
actual_ans = []
final_ans = []
labels=[]
#####FILE OPENED IN WRITE MODE#####
file1 = open('myfile.txt', 'w', encoding='utf-8')
def findingmostsimilar(poss , similar):
    besind=100
    best=[]
    for j in poss:
        for i in range(len(similar)):
            tok = similar[i].split(" ")
            if(j in tok):
                if(besind>=i):
                    besind=i
                    best.append(j)
    return best[0]
def ners(ner):
    possibles =[]
    for i in ner.keys():
        for j in ner[i]:
            possibles.append(j)
    return possibles
def find_max_freq(que_ner,ner,Top_similarity_sent):
    possibles = ners(ner)
    que_ners = ners(que_ner)
    for i in ner.keys():
        for j in ner[i]:
            possibles.append(j)
    words, counts = np.unique(possibles, return_counts=True)
    print("DATAFRAME WITH COUNT:")
    corpus_df = pd.DataFrame({'WORD':words, 'COUNT':counts})
    corpus_df.sort_values('COUNT', ascending=False, inplace=True)
    corpus_df.reset_index(drop=True,inplace=True)
    find=corpus_df[corpus_df.columns[0]]
    print(find)
    samefreq=[]
    maxfreq=0
    for i in range(len(find)):
        if(find[i].lower() not in que_ners):
            found=1
            freq=int(corpus_df[corpus_df['WORD']==find[i]]['COUNT'])
            if(maxfreq<=freq):
                samefreq.append(find[i])
                maxfreq=freq
            else:
                break

    if(len(samefreq)==1):
        return samefreq[0]
    elif(len(samefreq)>1):
        return findingmostsimilar(samefreq,Top_similarity_sent)
    return "NO ANSWER"
def cleanup_funct():
	Top_10_urls=[]
	whole_sent=[]
	consine_list=[]
	all_sen=[]
	Top_similarity_sent=[]
	ans=[]
	Sentence_NER=[]
	ner=[]
	quest=[]
	q_ner=[]
	q_text=[]
	Whole_output=[]
	final_answers=[]
	ss=[]
	label_ans=[]
	ans_type=[]
	all_sen1=[]
	random_ans=[]
	actual_ans=[]
	return (Top_10_urls,whole_sent,consine_list,all_sen,Top_similarity_sent,ans,
			Sentence_NER,ner,quest,q_ner,q_text,Whole_output,
			final_answers,label_ans,ss,ans_type,all_sen1, random_ans)
for line in input_file:
    #print(line)
    line=line.strip()
    line=line.split("@@@")
    labels.append(line[0])
    actual_ans.append(line[1].split(":::")[0])
    Questions.append(line[1].split(":::")[1])
start=time.time()
l=0        #initializing
for l in range(845,847): #running from 0 to 186
#qa="న్యూజిలాండ్ రాజధాని ఏమిటి?"
    qa=Questions[l]
    print("\n---------------------------------------------------------\n\n")
    print(" QUESTION ")
    print(l)
    #for qa in Questions:
    (Top_10_urls,whole_sent,consine_list,all_sen,Top_similarity_sent,ans,Sentence_NER,ner,quest,q_ner,q_text,Whole_output,final_answers,
    label_ans,ss,ans_type,all_sen1,random_ans ) = cleanup_funct()
    #answer=Question_Type(qa) # calling to get Rule Based Query Type
    print("\n---------------------------------------------------------\n\n")
    #print("Query Type:\n")
    #print(answer)
    q=translate(qa,'en','te') # translate te-->eng
    print("\n--------------------------------------------------------\n\n")
    print("Telugu Sentence:\t"+qa+"\n")
    print("English Sentence:\t"+q.text+"\n")
    print("\n\n")
    #print("Top 10 URLS:\n----------------------------------------------------------------\n\n")
    Top_10_urls = qquery(q.text) # calling get Top 10 URLS
    print("---------------------------------------------------------\n\n")
    Sentence_Extraction()  # calling to get whole content from 10 urls(well define sentences only)
    Finding_Similarity_Matrix(q.text) # calling find similarity matri between query and all sentences
    Top_similarity_sent = Most_Relative_Sentence() # calling to print higest top 5 similarity senteces.
    print("Top_similarity_sent : ",Top_similarity_sent)
    print("\n---------------------------------------------------------\n\n")
    qc=predict_ques(qa) # user query question classification
    #print("\n Question Classification Prediction for User Query:\n")
    print('qc :',qc)
    (ner,possibles) = NER_other_way(qc)
    print("\n------------------------------------------------------------\n\n")
    print("possible answers:")
    print(possibles)
    words, counts = np.unique(possibles, return_counts=True)
    print("DATAFRAME WITH COUNT:")
    corpus_df = pd.DataFrame({'WORD':words, 'COUNT':counts})
    corpus_df.sort_values('COUNT', ascending=False, inplace=True)
    corpus_df.reset_index(drop=True,inplace=True)
    find=corpus_df[corpus_df.columns[0]]
    print(find)
    dis = {'thrice':'three','twice':'two','once':'one','third':'three','second':'two','first':'one'}
    #lower = preprocess(q.text)
    (ner1,lower) = que_ner(qc, [q.text])
    print("QUESTION TOKENIZING TO NEGLECT FEW ANSWERS")
    print(lower)
    found=0
    if(len(find)==0):
        ans=find_max_freq(ner1,ner,Top_similarity_sent)
        print("FINAL ANSWER\n")
        print(ans)
        file1.write(ans+"\n")
        final_ans.append(ans)
    else:
        for i in range(len(find)):
            if(find[i] in dis.keys()):
                find[i]=dis[find[i]]
        samefreq=[]
        maxfreq=0
        for i in range(len(find)):
            if(find[i].lower() not in lower):
                found=1
                freq=int(corpus_df[corpus_df['WORD']==find[i]]['COUNT'])
                if(maxfreq<=freq):
                    samefreq.append(find[i])
                    maxfreq=freq
                else:
                    break

        if(len(samefreq)==1):
            print("FINAL ANSWER1:")
            print(samefreq[0])
            file1.write(samefreq[0]+"\n")
            final_ans.append(samefreq[0])
        elif(len(samefreq)>1):
            ans=findingmostsimilar(samefreq,Top_similarity_sent)
            print("FINAL ANSWER2:")
            print(ans)
            file1.write(ans+"\n")
            final_ans.append(ans)

        if(found==0):
            ans=find_max_freq(ner1,ner,Top_similarity_sent)
            print("FINAL ANSWER:")
            print(ans)
            file1.write(ans+"\n")
            final_ans.append(ans)
end=time.time()
print("\n-----------------------------------------------------------\n\n")
print("ACTUAL ANS\n")
print(actual_ans)
print("\n-----------------------------------------------------------\n\n")
print("LABELS\n")
print(labels)
print("\n-----------------------------------------------------------\n\n")
print("FINAL ANS\n")
print(final_ans)
print("TOTAL TIME FOR QUESTIONS in sec :")
print(end-start)
file1.close()


---------------------------------------------------------


 QUESTION 
845

---------------------------------------------------------



--------------------------------------------------------


Telugu Sentence:	మహిళల సింగిల్స్ ప్రపంచ బ్యాడ్మింటన్ ఛాంపియన్ షిప్ 2017 విజేత ఎవరు?

English Sentence:	Who is the winner of Women's Singles World Badminton Championship 2017?




BING SEARCH
Top 10 URLS:
----------------------------------------------------------------


https://en.wikipedia.org/wiki/2017_BWF_World_Championships_%E2%80%93_Women%27s_singles

https://en.wikipedia.org/wiki/2017_BWF_World_Championships

https://www.queryhome.com/sports/4986/has-2017-womens-singles-world-badminton-championships-wbc

https://www.queryhome.com/sports/5417/has-2017-womens-singles-national-badminton-championships

https://www.sportshistori.com/2017/08/bwf-world-championships-winners.html

https://www.youtube.com/watch?v=_JyL12e-a80

https://en.wikipedia.org/wiki/Gold_medalists_at_the_BWF_World_Champion